In [1]:
"""
POC 1 - Source Based Query Routing

Using two different knowledge sources:
1. Product Data from RAG POC 1
2. Meeting Transcript from POC 2

Router decides which agent should answer.
"""

from langchain_ollama import ChatOllama
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate


# using local llama3 model
llm = ChatOllama(model="llama3", temperature=0)


# ---------------------------
# Product Agent Setup
# ---------------------------
product_loader = TextLoader("products.txt")
product_docs = product_loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

product_chunks = splitter.split_documents(product_docs)

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

product_db = FAISS.from_documents(product_chunks, embedding_model)
product_retriever = product_db.as_retriever(search_kwargs={"k": 3})


# ---------------------------
# Meeting Agent Setup
# ---------------------------
meeting_loader = TextLoader("meeting_transcript.txt")
meeting_docs = meeting_loader.load()

meeting_chunks = splitter.split_documents(meeting_docs)

meeting_db = FAISS.from_documents(meeting_chunks, embedding_model)
meeting_retriever = meeting_db.as_retriever(search_kwargs={"k": 3})


# ---------------------------
# Prompt
# ---------------------------
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer only from the given context."),
        ("human", "Context:\n{context}\n\nQuestion:\n{question}")
    ]
)


# ---------------------------
# Router
# ---------------------------
router_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Classify the query into either 'product' or 'meeting'. "
            "Respond with only one word."
        ),
        ("human", "{query}")
    ]
)


def route_query(query):
    chain = router_prompt | llm
    decision = chain.invoke({"query": query}).content.strip().lower()
    return decision


# ---------------------------
# Main Handler
# ---------------------------
def handle_query(query):

    source = route_query(query)

    if source == "product":
        docs = product_retriever.invoke(query)
    else:
        docs = meeting_retriever.invoke(query)

    context = "\n\n".join([doc.page_content for doc in docs])

    chain = prompt | llm
    response = chain.invoke(
        {
            "context": context,
            "question": query
        }
    )

    return response.content


# ---------------------------
# Testing
# ---------------------------
if __name__ == "__main__":

    questions = [
        "Suggest a good laptop",
        "What were the action items in the meeting?"
    ]

    for q in questions:
        print("\nQuestion:", q)
        print("Answer:", handle_query(q))


C:\Users\Shruti Mishra\AppData\Local\Temp\ipykernel_10776\2768058838.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Question: Suggest a good laptop
Answer: Based on the given context, I would suggest the Apple MacBook Air M2 (₹1,15,000) or the Dell XPS 13 (₹1,20,000). Both are high-performance laptops with long battery life. The choice between the two depends on your personal preference for operating system and processor type.

Question: What were the action items in the meeting?
Answer: According to the context, the action items from the meeting are:

1. Backend Engineer will optimize APIs by next week.
2. Frontend Engineer will create new UI mockups.
3. QA Lead will prepare test cases.
